In [36]:
import numpy as np
import pandas as pd
import pickle
import glob
from tqdm import notebook

In [42]:
def get_labels(data_list, N = 3, hw_ratio = 720/1280, return_type = 'list', return_grids = False):
    """
    data_list: list[tuple], in which tuple[0] is the frame_idx, tuple[1] is a (33,3) np array
    grid: int, number of grids. E.g., grid = 3 results in a 3 by 3 split
    return_type: 'list' | 'df'
    """
    assert isinstance(data_list, list)

    results = []
    grids = []
    for frame_idx, data in data_list:
        nose_x = data[0][0]
        l_shoulder_x = data[11][0]
        r_shoulder_x = data[12][0]
        l_hip_x = data[23][0]
        r_hip_x = data[24][0]

        # y1, y2 = 0.001, 0.999 # This is the simplest method
        l_eye_y = data[2][1]
        r_eye_y = data[5][1]
        mid_eye_y = (l_eye_y + r_eye_y) / 2
        nose_y = data[0][1]
        nose_eye_diff = nose_y - mid_eye_y
        y1 = max(0.001, mid_eye_y - nose_eye_diff * 2) # x2 because: https://www.artyfactory.com/portraits/pencil-portraits/proportions-of-a-head.html
        l_hip_y = data[23][1]
        r_hip_y = data[24][1]
        mid_hip_y = (l_hip_y + r_hip_y) / 2
        y2 = min(0.999, mid_hip_y)

        grids_height = y2 - y1
        grids_width = grids_height * hw_ratio # normalized by hw_ratio

        xc = (nose_x + (l_shoulder_x + r_shoulder_x)/2 + (l_hip_x + r_hip_x)/2) / 3
        # print(xc)
        x1 = xc - 0.5 * grids_width + 0.001
        x2 = xc + 0.5 * grids_width - 0.001
        if x1 <= 0.0 or x2 >= 1.0:
            results.append((frame_idx, None, None, None))
            grids.append((frame_idx, None))
            continue


        # Compute grids for current frame
        if return_grids:
            cur_grid = [x1]
            for i in range(1,N):
                cur_grid.append(x1 + (x2-x1) * i/N)
            cur_grid.append(x2)
            cur_grid.append(y1)
            for j in range(1,N):
                cur_grid.append(y1 + (y2-y1) * j/N)
            cur_grid.append(y2)
            grids.append((frame_idx, tuple(cur_grid)))

        # l_wrist_x, l_wrist_y = data[15][0], data[15][1]
        # r_wrist_x, r_wrist_y = data[16][0], data[16][1]
        l_pinky_x, l_pinky_y = data[17][0], data[17][1]
        r_pinky_x, r_pinky_y = data[18][0], data[18][1]
        l_index_x, l_index_y = data[19][0], data[19][1]
        r_index_x, r_index_y = data[20][0], data[20][1]
        l_thumb_x, l_thumb_y = data[21][0], data[21][1]
        r_thumb_x, r_thumb_y = data[22][0], data[22][1]
        l_hand_x = (l_pinky_x + l_index_x + l_thumb_x) / 3
        l_hand_y = (l_pinky_y + l_index_y + l_thumb_y) / 3
        r_hand_x = (r_pinky_x + r_index_x + r_thumb_x) / 3
        r_hand_y = (r_pinky_y + r_index_y + r_thumb_y) / 3

        l_col = np.floor(min(max(l_hand_x - x1, 0), x2-x1) / grids_width * N) + 1
        r_col = np.floor(min(max(r_hand_x - x1, 0), x2-x1) / grids_width * N) + 1
        l_row = np.floor(min(max(l_hand_y - y1, 0), y2-y1) * N) + 1
        r_row = np.floor(min(max(r_hand_y - y1, 0), y2-y1) * N) + 1

        l_label = int((l_row - 1)*N + l_col)
        r_label = int((r_row - 1)*N + r_col)
        label = (l_label - 1)*N*N + r_label

        # Filter the labels that are out of the range
        if (l_label >= 1 and l_label <= N*N) and (r_label >= 1 and r_label <= N*N) and (label >= 1 and label <= N*N*N*N):
            results.append((frame_idx, l_label, r_label, label))
        else:
            results.append((frame_idx, None, None, None))

    if return_grids:
        assert len(results) == len(grids)
    if return_type == 'list':
        if return_grids:
            return results, grids
        else:
            return results
    elif return_type == 'df':
        results_cleaned = list(filter(lambda x: x[1] != None, results))
        data = np.array(results_cleaned, dtype=[('frame_idx', 'i4'), ('l_label', 'i4'), ('r_label', 'i4'), ('label', 'i4')])
        df = pd.DataFrame.from_records(data)
        if return_grids:
            return df, grids
        else:
            return df


In [9]:
# Load keypoints data for testing
kp_file = '1FYADI0rSCc.pkl'
with open(kp_file, 'rb') as f:
    kp_data = pickle.load(f)

print(len(kp_data))
print(type(kp_data[0]))
print(len(kp_data[0]))
print(list(map(type, kp_data[0])))

10631
<class 'tuple'>
2
[<class 'int'>, <class 'numpy.ndarray'>]


In [13]:
df = get_labels(kp_data, return_type='df')
print(max(df['label']))
print(min(df['label']))

81
1


In [15]:
kp_data[0][1]

array([[ 0.51772863,  0.26019549, -0.3459377 ],
       [ 0.52931899,  0.20547014, -0.323264  ],
       [ 0.5405513 ,  0.20125471, -0.32332459],
       [ 0.54862118,  0.19822794, -0.32326761],
       [ 0.49627286,  0.21791254, -0.31958356],
       [ 0.48759741,  0.22288306, -0.31954843],
       [ 0.48062468,  0.22886366, -0.3199102 ],
       [ 0.56738573,  0.21134423, -0.16489023],
       [ 0.47452125,  0.25545484, -0.14034304],
       [ 0.54495466,  0.30404788, -0.28497794],
       [ 0.50603056,  0.31791252, -0.27835807],
       [ 0.67660141,  0.48135298, -0.05196721],
       [ 0.43467182,  0.50961423, -0.06757601],
       [ 0.74118978,  0.73173457, -0.10129409],
       [ 0.36590484,  0.79860914, -0.20265391],
       [ 0.76259792,  0.81234944, -0.43712729],
       [ 0.39123148,  0.91132933, -0.6221174 ],
       [ 0.7870152 ,  0.87686443, -0.51381725],
       [ 0.40723735,  0.9578681 , -0.69321555],
       [ 0.76200616,  0.8670181 , -0.53664333],
       [ 0.42895013,  0.91248846, -0.711

In [21]:
# Generate N samples for testing,
# Each sample is a numpy array of shape (33,3), and the range of each element is from a normal distribution with mean 0.5 and std 1
n_samples = 100000
samples = np.random.normal(0.5, 1, (n_samples, 33, 3))
samples_list = [(i, samples[i]) for i in range(n_samples)]
print(len(samples_list))
print(samples_list[0][1].shape)

100000
(33, 3)


In [28]:
df_samples = get_labels(samples_list, return_type='df')
print(max(df_samples['label']))
print(min(df_samples['label']))

# number of rows in df where label > 81 or label < 1
print(len(df_samples[(df_samples['label'] > 81) | (df_samples['label'] < 1)]))

# number of rows in df where l_label > 9 or l_label < 1
print(len(df_samples[(df_samples['l_label'] > 9) | (df_samples['l_label'] < 1)]))

3
1
0
0


In [29]:
df_samples[(df_samples['label'] > 81) | (df_samples['label'] < 1)]

,frame_idx,l_label,r_label,label


In [30]:
df_samples

,frame_idx,l_label,r_label,label
0,3,1,1,1
1,10,1,1,1
2,13,1,1,1
3,20,1,1,1
4,30,1,3,3
...,...,...,...,...
10947,99948,1,1,1
10948,99952,1,1,1
10949,99960,1,1,1
10950,99967,1,1,1


In [43]:
# Test all key points files
kp_files = glob.glob('../data/videos_kp/*.pkl')

df_list = []
for kp_file in notebook.tqdm(kp_files):
    with open(kp_file, 'rb') as f:
        kp_data = pickle.load(f)
    df = get_labels(kp_data, return_type='df')
    df_list.append(df)
    # print(kp_file)
    # print(max(df['label']))
    # print(min(df['label']))
    # print(len(df[(df['label'] > 81) | (df['label'] < 1)]))
    # print(len(df[(df['l_label'] > 9) | (df['l_label'] < 1)]))
    # print(len(df[(df['r_label'] > 9) | (df['r_label'] < 1)]))
    # print()
df_all = pd.concat(df_list)
print(max(df_all['label']))
print(min(df_all['label']))

  0%|          | 0/71 [00:00<?, ?it/s]

81
1


In [34]:
df_all.head(10)

,frame_idx,l_label,r_label,label
0,179,1,2,2
1,2745,1,1,1
2,3240,1,1,1
3,3639,1,1,1
4,3640,1,1,1
5,3641,1,1,1
6,3642,1,1,1
7,3643,1,1,1
8,3644,1,1,1
9,3645,1,1,1


In [44]:
print(len(df_all[(df_all['label'] > 81) | (df_all['label'] < 1)]))
df_all[(df_all['label'] > 81) | (df_all['label'] < 1)]

0


,frame_idx,l_label,r_label,label


In [40]:
print(len(df_all[(df_all['l_label'] > 9) | (df_all['l_label'] < 1)]))

728


In [41]:
print(len(df_all[(df_all['r_label'] > 9) | (df_all['r_label'] < 1)]))

728
